### **Curating bcc_face_forehead2.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12887269.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 932 × 20248
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.14445373, -0.36695725,  0.07438989, ..., -0.06124804,
        -0.4597496 , -0.21085592],
       [-0.14445373,  3.29608   ,  1.3681718 , ..., -0.06124804,
        -0.4597496 ,  2.2533107 ],
       [-0.14445373, -0.36695725, -0.44497144, ..., -0.06124804,
        -0.4597496 , -0.21085592],
       ...,
       [-0.14445373, -0.36695725, -0.44497144, ..., -0.06124804,
        -0.4597496 , -0.21085592],
       [-0.14445373, -0.36695725, -0.44497144, ..., -0.06124804,
        -0.4597496 , -0.21085592],
       [-0.14445373, -0.36695725,  1.2703657 , ..., -0.06124804,
        -0.4597496 , -0.21085592]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<932x20248 sparse matrix of type '<class 'numpy.float32'>'
	with 18871136 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.14445373
  (0, 1)	-0.36695725
  (0, 2)	0.07438989
  (0, 3)	-0.08825809
  (0, 4)	0.39757022
  (0, 5)	1.119182
  (0, 6)	-0.34923625
  (0, 7)	0.6836645
  (0, 8)	-0.49735713
  (0, 9)	0.16835248
  (0, 10)	-0.07162703
  (0, 11)	-0.3854275
  (0, 12)	0.40565795
  (0, 13)	0.30552384
  (0, 14)	1.3275654
  (0, 15)	-0.16839464
  (0, 16)	0.077364095
  (0, 17)	0.015692916
  (0, 18)	-0.10365729
  (0, 19)	-0.27176544
  (0, 20)	0.79115236
  (0, 21)	0.30871102
  (0, 22)	-0.2902589
  (0, 23)	-0.42163387
  (0, 24)	-0.08611214
  :	:
  (931, 20223)	-0.38523424
  (931, 20224)	-0.16575773
  (931, 20225)	-0.6647105
  (931, 20226)	-0.056728616
  (931, 20227)	-1.4170445
  (931, 20228)	-1.1808784
  (931, 20229)	-0.61031246
  (931, 20230)	-1.0569696
  (931, 20231)	-0.3310364
  (931, 20232)	-0.90677345
  (931, 20233)	-1.103884
  (931, 20234)	-1.2669826
  (931, 20235)	-0.16062994
  (931, 20236)	-0.9556201
  (931, 20237)	-0.40020573
  (931, 20238)	-0.2291935
  (931, 20239)	-0.96642184
  (931, 20240)	-0.06

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12887269/filtered_feature_bc_matrix.h5')

/home/jovyan/my-conda-envs/shi_env/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 932 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<932x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 5452505 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 16)	1.0
  (0, 18)	1.0
  (0, 22)	1.0
  (0, 24)	7.0
  (0, 26)	1.0
  (0, 29)	10.0
  (0, 30)	12.0
  (0, 32)	10.0
  (0, 36)	2.0
  (0, 39)	1.0
  (0, 42)	1.0
  (0, 43)	3.0
  (0, 44)	13.0
  (0, 45)	1.0
  (0, 48)	4.0
  (0, 51)	5.0
  (0, 52)	2.0
  (0, 53)	6.0
  (0, 57)	2.0
  (0, 58)	36.0
  (0, 59)	21.0
  (0, 60)	12.0
  (0, 61)	2.0
  (0, 62)	15.0
  (0, 68)	2.0
  :	:
  (931, 36406)	5.0
  (931, 36407)	4.0
  (931, 36408)	4.0
  (931, 36409)	1.0
  (931, 36410)	2.0
  (931, 36411)	2.0
  (931, 36412)	3.0
  (931, 36413)	3.0
  (931, 36415)	2.0
  (931, 36416)	1.0
  (931, 36422)	1.0
  (931, 36429)	6.0
  (931, 36434)	1.0
  (931, 36559)	40.0
  (931, 36560)	32.0
  (931, 36561)	105.0
  (931, 36562)	122.0
  (931, 36563)	1.0
  (931, 36564)	67.0
  (931, 36565)	99.0
  (931, 36566)	34.0
  (931, 36567)	3.0
  (931, 36568)	58.0
  (931, 36569)	9.0
  (931, 36571)	45.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
LINC01409
LINC01128
LINC00115
FAM41C
...
AC011043.1
AL592183.1
AC240274.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
LINC00115,LINC00115,ENSG00000225880
FAM41C,FAM41C,ENSG00000230368
...,...,...
AC011043.1,AC011043.1,ENSG00000276256
AL592183.1,AL592183.1,ENSG00000273748
AC240274.1,AC240274.1,ENSG00000271254
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

17

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
ENSG00000225880,LINC00115,ENSG00000225880
ENSG00000230368,FAM41C,ENSG00000230368
...,...,...
ENSG00000276256,AC011043.1,ENSG00000276256
ENSG00000273748,AL592183.1,ENSG00000273748
ENSG00000271254,AC240274.1,ENSG00000271254


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000276256
ENSG00000273748
ENSG00000271254


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

116184

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

20194

In [57]:
len(var_to_keep_araw)

36503

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000276256
ENSG00000273748
ENSG00000271254


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

16309

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.017832,0.037396,0.008977,0.186887,0.000924,5,1.640862,0.431584,0.000066,1.013806
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,0.077513,0.028362,0.155181,0.078314,2.026988,3,8.836067,5.791462,1.837932,0.014921
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.038690,0.118446,0.017786,0.075171,0.005046,5,1.284307,0.644707,0.000034,0.062423
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.019754,0.016019,0.012933,0.003784,0.007279,6,1.647192,5.630633,0.000509,0.005931
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,0.503677,0.390177,0.137865,0.011079,1.166356,8,7.311481,1.087696,0.154580,0.646512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0.081630,0.062573,0.046882,0.017244,1.475838,0,6.653639,4.916734,0.219937,0.085883
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.013976,0.066107,0.040095,0.150801,0.002337,5,1.342501,1.433669,0.000710,0.006274
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.051190,0.068114,0.122250,0.058599,0.656767,4,2.236353,3.389602,0.382951,0.045885


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.037396,0.008977,0.186887,0.000924,5,1.640862,0.431584,0.000066,1.013806,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,0.028362,0.155181,0.078314,2.026988,3,8.836067,5.791462,1.837932,0.014921,EFO:0010961
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.118446,0.017786,0.075171,0.005046,5,1.284307,0.644707,0.000034,0.062423,EFO:0010961
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.016019,0.012933,0.003784,0.007279,6,1.647192,5.630633,0.000509,0.005931,EFO:0010961
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,0.390177,0.137865,0.011079,1.166356,8,7.311481,1.087696,0.154580,0.646512,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0.062573,0.046882,0.017244,1.475838,0,6.653639,4.916734,0.219937,0.085883,EFO:0010961
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.066107,0.040095,0.150801,0.002337,5,1.342501,1.433669,0.000710,0.006274,EFO:0010961
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.068114,0.122250,0.058599,0.656767,4,2.236353,3.389602,0.382951,0.045885,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.186887,0.000924,5,1.640862,0.431584,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,0.078314,2.026988,3,8.836067,5.791462,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.075171,0.005046,5,1.284307,0.644707,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.003784,0.007279,6,1.647192,5.630633,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,0.011079,1.166356,8,7.311481,1.087696,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0.017244,1.475838,0,6.653639,4.916734,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.150801,0.002337,5,1.342501,1.433669,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.058599,0.656767,4,2.236353,3.389602,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:0000057',
 'CL:4033013',
 'CL:0002187',
 'CL:0000786',
 'CL:0002139',
 'CL:0002573',
 'CL:0000815',
 'CL:0000235',
 'CL:0000669',
 'CL:0000990',
 'CL:0000646',
 'CL:0000192',
 'CL:0000784']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.000924,5,1.640862,0.431584,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,2.026988,3,8.836067,5.791462,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.005046,5,1.284307,0.644707,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.007279,6,1.647192,5.630633,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,1.166356,8,7.311481,1.087696,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,1.475838,0,6.653639,4.916734,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.002337,5,1.342501,1.433669,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.656767,4,2.236353,3.389602,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187


In [96]:
araw.obs

""
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACCGGGTAGGTACC-1
AAACCTCATGAAGTTG-1
AAACGAGACGGTTGAT-1
...
TTGTACACCTCGAACA-1
TTGTGAGGCATGACGC-1
TTGTGGTAGGAGGGAT-1
TTGTTGTGTGTCAAGA-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['WSSKNKCLsp12887269'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['WSSKNKCLsp12887269']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,5,1.640862,0.431584,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,3,8.836067,5.791462,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,5,1.284307,0.644707,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,6,1.647192,5.630633,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,8,7.311481,1.087696,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0,6.653639,4.916734,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,5,1.342501,1.433669,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,4,2.236353,3.389602,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [111]:
mapping = dict(zip(suppl_info['donor'], suppl_info['development_stage_ontology_term_id']))

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [113]:
# change datatype of the column

In [114]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [115]:
# view unique values of development_stage_ontology_term_id column

In [116]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000158']

In [117]:
# view adata.obs

In [118]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,1.640862,0.431584,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,8.836067,5.791462,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,1.284307,0.644707,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,1.647192,5.630633,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,7.311481,1.087696,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,6.653639,4.916734,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,1.342501,1.433669,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,2.236353,3.389602,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158


#### **disease_ontology_term_id**

In [119]:
# Assign normal since all are healthy patients

In [120]:
# add the disease_ontology_term_id column

In [121]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [122]:
mapping = dict(zip(suppl_info['donor'], suppl_info['disease_ontology_term_id']))

In [123]:
adata.obs['disease_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [124]:
#change data type of column

In [125]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [126]:
# view obs

In [127]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.431584,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,5.791462,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.644707,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,5.630633,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,1.087696,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,4.916734,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,1.433669,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,3.389602,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804


#### **is_primary_data**

In [128]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [129]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,0.000066,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,1.837932,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.000034,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.000509,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,0.154580,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0.219937,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.000710,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.382951,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True


In [130]:
#change data type of column

In [131]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [132]:
# assign organism id 

In [133]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [134]:
#change data type of column

In [135]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [136]:
# view obs

In [137]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,1.013806,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,0.014921,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,0.062423,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,0.005931,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,0.646512,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,0.085883,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,0.006274,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,0.045885,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [138]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [139]:
# change data type

In [140]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [141]:
# view obs

In [142]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,EFO:0010961,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,EFO:0010961,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown


In [143]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [144]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [145]:
mapping = dict(zip(suppl_info['donor'], suppl_info['sex_ontology_term_id']))

In [146]:
adata.obs['sex_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [147]:
# change data type

In [148]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,c2l_POSTN+ fibroblasts,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,c2l_Suprabasal keratinocytes,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,c2l_POSTN+ fibroblasts,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,c2l_Basal keratinocytes,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,c2l_Suprabasal keratinocytes,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,c2l_Suprabasal keratinocytes,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,c2l_POSTN+ fibroblasts,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,c2l_Basal keratinocytes,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383


#### **suspension_type**

In [150]:
# since visium suspension type is 'na'

In [151]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [152]:
# change data type

In [153]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [154]:
# view obs

In [155]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,5.555018,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,8.836067,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,6.393409,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,5.630633,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,7.311481,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,6.653639,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,5.124152,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,3.389602,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na


#### **tissue_ontology_term_id**

In [156]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [157]:
mapping = dict(zip(suppl_info['donor'], suppl_info['tissue_ontology_term_id']))

In [158]:
adata.obs['tissue_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [159]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [160]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0016475']

In [161]:
# view obs

In [162]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,10808,78445,903,1.151125,9594,12.230225,AAACAGCTTTCAGAAG-1,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACAGGGTCTATATT-1,1,47,13,8325,38437,3302,8.590681,6258,16.281187,AAACAGGGTCTATATT-1,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCGGGTAGGTACC-1,1,42,28,9388,48289,533,1.103771,5811,12.033796,AAACCGGGTAGGTACC-1,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCTCATGAAGTTG-1,1,37,19,8348,33269,535,1.608104,4755,14.292584,AAACCTCATGAAGTTG-1,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACGAGACGGTTGAT-1,1,35,79,7861,33543,1025,3.055779,6164,18.376413,AAACGAGACGGTTGAT-1,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,51,1,2058,4267,173,4.054371,605,14.178580,TTGTACACCTCGAACA-1,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGAGGCATGACGC-1,1,43,13,9545,51974,669,1.287182,5958,11.463424,TTGTGAGGCATGACGC-1,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGGTAGGAGGGAT-1,1,50,28,1202,2179,169,7.755851,330,15.144561,TTGTGGTAGGAGGGAT-1,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475


In [163]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [164]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [165]:
adata.obsm

AxisArrays with keys: X_spatial

In [166]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [167]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([591.54016 , 171.27043 , 126.74585 , 105.1378  ,  80.48521 ,
        63.40362 ,  56.09357 ,  52.058117,  48.38504 ,  45.034225,
        44.207047,  43.40581 ,  43.0433  ,  42.86656 ,  42.6923  ,
        42.406403,  42.197674,  41.998512,  41.82618 ,  41.761505,
        41.641346,  41.59494 ,  41.50343 ,  41.4254  ,  41.208714,
        41.070824,  40.990257,  40.795994,  40.56169 ,  40.504734,
        40.42256 ,  40.372158,  40.23513 ,  40.12877 ,  40.118046,
        39.984207,  39.827175,  39.75626 ,  39.684837,  39.437717,
        39.33667 ,  39.255264,  39.216045,  39.103718,  39.006935,
        38.926983

In [168]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the forehead stained with H&E'

In [169]:
adata.uns['title'] = 'Visium spatial - bcc_face_forehead2'

In [170]:
adata.uns['default_embedding'] = 'X_spatial'

In [171]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [172]:
adata

AnnData object with n_obs × n_vars = 932 × 36503
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_on

In [173]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [174]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [175]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [176]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,10808,78445,903,1.151125,9594,12.230225,0.101487,0.172054,0.001321,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACAGGGTCTATATT-1,1,8325,38437,3302,8.590681,6258,16.281187,0.047443,0.082926,0.491657,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCGGGTAGGTACC-1,1,9388,48289,533,1.103771,5811,12.033796,0.022835,0.242387,0.002502,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCTCATGAAGTTG-1,1,8348,33269,535,1.608104,4755,14.292584,0.023102,0.078664,0.001047,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACGAGACGGTTGAT-1,1,7861,33543,1025,3.055779,6164,18.376413,0.069110,0.633398,1.552613,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,2058,4267,173,4.054371,605,14.178580,0.038040,0.295172,0.097615,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGAGGCATGACGC-1,1,9545,51974,669,1.287182,5958,11.463424,0.043683,0.014097,0.000097,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGGTAGGAGGGAT-1,1,1202,2179,169,7.755851,330,15.144561,0.045594,0.334719,0.663829,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475


In [177]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [178]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [179]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,10808,78445,903,1.151125,9594,12.230225,0.101487,0.172054,0.001321,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACAGGGTCTATATT-1,1,8325,38437,3302,8.590681,6258,16.281187,0.047443,0.082926,0.491657,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCGGGTAGGTACC-1,1,9388,48289,533,1.103771,5811,12.033796,0.022835,0.242387,0.002502,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACCTCATGAAGTTG-1,1,8348,33269,535,1.608104,4755,14.292584,0.023102,0.078664,0.001047,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
AAACGAGACGGTTGAT-1,1,7861,33543,1025,3.055779,6164,18.376413,0.069110,0.633398,1.552613,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTACACCTCGAACA-1,1,2058,4267,173,4.054371,605,14.178580,0.038040,0.295172,0.097615,...,CL:4033013,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGAGGCATGACGC-1,1,9545,51974,669,1.287182,5958,11.463424,0.043683,0.014097,0.000097,...,CL:0000057,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475
TTGTGGTAGGAGGGAT-1,1,1202,2179,169,7.755851,330,15.144561,0.045594,0.334719,0.663829,...,CL:0002187,WSSKNKCLsp12887269,HsapDv:0000158,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000383,na,UBERON:0016475


In [180]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [181]:
#check the format of expression matrix

In [182]:
adata.X

<932x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 18820808 stored elements in Compressed Sparse Row format>

In [183]:
araw.X

<932x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 5449498 stored elements in Compressed Sparse Row format>

In [184]:
#Copy raw counts to adata.raw

In [185]:
adata.raw = araw

In [186]:
#write the curated object to final_objects folder

In [187]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/bcc_face_forehead2.h5ad', compression = 'gzip')